In [63]:
import nltk
import matplotlib.pyplot as plt
import numpy
from nltk.corpus import twitter_samples
import random
import numpy as np
import math as m

nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('tokenize')


[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Adi\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Adi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading tokenize: Package 'tokenize' not found in
[nltk_data]     index


False

In [64]:
# use predefined function ‘fileids()’ to see the content
twitter_samples.fileids()
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [65]:
# print positive in greeen
print('\033[92m' + all_positive_tweets[random.randint(0,5000)])

# print negative in red
print('\033[91m' + all_negative_tweets[random.randint(0,5000)])

@Mehdi_Mustafa @Ali_SyedRaza 
ROFL not going to happen, rest assured :D
I thought finding someone was hard but finding them and trying to make them stay is harder :(

#imintoher


In [66]:

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings
import re
import string                              # for string operations

stop_words = stopwords.words('english')
punctuation = string.punctuation

tweets = all_positive_tweets + all_negative_tweets ## Concatenate the lists. 
labels = np.append(np.ones((len(all_positive_tweets),1)), np.zeros((len(all_negative_tweets),1)), axis = 0)

train_pos  = all_positive_tweets[:4000]
train_neg  = all_negative_tweets[:4000]
train_x = train_pos + train_neg 
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)), axis=0).reshape(-1)


test_pos  = all_positive_tweets[1000:]
test_neg  = all_negative_tweets[1000:]
test_x = test_pos + test_neg
test_y = np.append(np.ones(len(test_pos)),np.zeros(len(test_neg)), axis=0)



In [67]:
print(train_y)

[1. 1. 1. ... 0. 0. 0.]


In [68]:
def preprocess(tweet) :
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    stemmer = PorterStemmer()
    tweet_tokens = tokenizer.tokenize(tweet)
    cleaned_stemmed_tweet = []
    for token in tweet_tokens:
        if token not in stop_words and token not in punctuation :
            cleaned_stemmed_tweet.append(stemmer.stem(token))
    return cleaned_stemmed_tweet




In [69]:
tweet_processed = preprocess(all_positive_tweets[1456])
print(tweet_processed)

['u', 'cant', 'chang', 'peopl', 'feel', 'u', 'dnt', 'tri', 'live', 'ur', 'life', 'happi', ':)']


In [70]:
positive_dictionary = {}
negative_dictionary = {}

for i in range(len(all_positive_tweets)) :
    pos_tokens = preprocess(all_positive_tweets[i])
    neg_tokens = preprocess(all_negative_tweets[i])
    for token in pos_tokens :
        if token in positive_dictionary :
            positive_dictionary[token]+=1
        else :
            positive_dictionary[token]=1
    for token in neg_tokens :
        if token in negative_dictionary :
            negative_dictionary[token]+=1
        else :
            negative_dictionary[token]=1

In [71]:
def sentiment(tweet) :
    if tweet in all_negative_tweets:
        return 0
    return 1

In [72]:
def Vectorize(tweets, bias) :
    freq_table = [[]]
    for tweet in tweets:    
        toks = preprocess(tweet)
        negative_score = 0
        positive_score = 0
        for tok in toks:
            if tok in negative_dictionary :
                negative_score+=negative_dictionary[tok]
            if tok in positive_dictionary:
                positive_score+=positive_dictionary[tok]
        if len(toks)>0:
            freq_table.append((np.array([bias,positive_score,negative_score])))
    freq_table= freq_table[1:]
    return np.array(freq_table)




In [73]:
def sigmoid(x, w):
    sig = []
    clip_value = 10  
    for value in x:
        z = np.dot(value, w) 
        z_clipped = np.clip(z, -clip_value, clip_value)
        sig_value = 1.0 / (1 + np.exp(-z_clipped))
        sig.append(sig_value)
    return np.array(sig)

In [74]:
from sklearn.metrics import log_loss

In [75]:
def train(x_train, y_train, learning_rate, epochs):
    x_train_processed = Vectorize(x_train, 1)  
    weight = np.zeros(3)

    for epoch in range(epochs+1):
        sig = sigmoid(x_train_processed, weight)
        loss = log_loss(y_train, sig)  
        
        # we use the dot product of x transpose in this situation in order to aggregate the erros
        # meaning that we use this in order to see how much each feature is corresponding to error and adjust accordingly
        gradient = np.dot(x_train_processed.T, (sig - y_train)) / len(y_train)
        weight -= learning_rate * gradient
        
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss}')
            print(weight)
    
    return weight

In [76]:
weights = train(train_x,train_y,0.01,1000)

Epoch 0, Loss: 0.6931471805599454
[  0.           7.67132438 -10.83586812]
Epoch 100, Loss: 0.05380451807778632
[ 3.60093532e-03  8.12504401e+00 -8.73426842e+00]
Epoch 200, Loss: 0.048563505419131035
[ 5.40354262e-03  7.37899846e+00 -7.68161702e+00]
Epoch 300, Loss: 0.04835419074449679
[ 0.00690303  6.49537431 -6.76040971]
Epoch 400, Loss: 0.04824862446026552
[ 0.00839992  5.611057   -5.83987276]
Epoch 500, Loss: 0.04815211106119109
[ 0.00989522  4.72672811 -4.91935457]
Epoch 600, Loss: 0.04785769137222982
[ 0.01138915  3.84240224 -3.99884635]
Epoch 700, Loss: 0.04755400561889181
[ 0.01288289  2.95809305 -3.07834452]
Epoch 800, Loss: 0.047261201802485886
[ 0.01437939  2.07381784 -2.15785137]
Epoch 900, Loss: 0.046023274571399
[ 0.01588388  1.18960283 -1.23739229]
Epoch 1000, Loss: 0.04247813627540391
[ 0.01741819  0.30526287 -0.31754746]


In [77]:
sigmoids = sigmoid(Vectorize(test_x,1),weights)
correct = 0
for i in range(len(test_x)):
    if test_y[i] == int(sigmoids[i] > 0.5) :
        correct+=1
print(correct/len(sigmoids))

0.995125


In [78]:
print(weights)

[ 0.01741819  0.30526287 -0.31754746]
